In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator
from embeddings_tools import get_embeddings
from intelligence_tools import AnalyzeVariables

In [3]:
df = pd.read_csv('../datasets/2024_ClassDim_project_bonus_data_train.csv')


In [10]:
av = AnalyzeVariables(df)

In [14]:
av.shape()

'3750 - строк и 22 колонок с переменными в датасете'

In [18]:
av.detect_outliers()

{'feature1': 1,
 'feature2': 273,
 'feature3': 273,
 'feature4': 273,
 'feature5': 85,
 'feature7': 359,
 'feature10': 523,
 'feature11': 3,
 'feature12': 64,
 'feature13': 95,
 'feature15': 173,
 'feature16': 1,
 'feature18': 117,
 'feature19': 446,
 'feature21': 10,
 'Class': 595}

In [20]:
av.missing_percentage()

feature1     0.0
feature2     0.0
feature3     0.0
feature4     0.0
feature5     0.0
feature6     0.0
feature7     0.0
feature8     0.0
feature9     0.0
feature10    0.0
feature11    0.0
feature12    0.0
feature13    0.0
feature14    0.0
feature15    0.0
feature16    0.0
feature17    0.0
feature18    0.0
feature19    0.0
feature20    0.0
feature21    0.0
Class        0.0
dtype: float64

In [5]:
bdf = BaseDatasetTransform(df,target='Class')

In [6]:
X_train, X_test, y_train, y_test,categorical_features = bdf.fit_transform()

Series([], dtype: int64)

В колонке(ах) ['Class'] нет пропущенных значений
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index([], dtype='object')

Числовые колонки:
Index(['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
       'feature7', 'feature8', 'feature9', 'feature10', 'feature11',
       'feature12', 'feature13', 'feature14', 'feature15', 'feature16',
       'feature17', 'feature18', 'feature19', 'feature20', 'feature21'],
      dtype='object')


In [7]:
lgbm_model = lgb.LGBMClassifier(verbose=-1,is_unbalance=True)

columns_to_delete = []  


model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='binary',
    metric='binary_error'
)

In [8]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=[],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(is_unbalance=True,
                                                                 metric='binary_error',
                                                                 objective='binary',
                                                                 verbose=-1)))

In [9]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.944889
1,precision,0.884189
2,recall,0.921645
3,f1,0.901430
